# **LLMs Locales en la Nube: Ollama + RAG con LangChain**

**Objetivo de la clase:**
Aprender a levantar un servidor de LLMs en Google Colab (GPU gratuita),
exponerlo al exterior con ngrok, y construir un sistema RAG (Retrieval-Augmented
Generation) con LangChain que responda preguntas sobre documentos propios.

## ¿Qué es RAG y por qué importa?

Un LLM "de base" solo conoce lo que vio durante su entrenamiento.
RAG resuelve esto en 3 pasos:

  1. 📄 **Indexar** tus documentos (convertirlos en vectores)
  2. 🔍 **Recuperar** los fragmentos más relevantes para cada pregunta
  3. 💬 **Generar** una respuesta usando esos fragmentos como contexto

```
Pregunta ──► Recuperador ──► Fragmentos relevantes ──► LLM ──► Respuesta
                  ▲
            Base vectorial
            (tus documentos)
```

## Requisitos previos
- Cuenta Google con acceso a Colab
- Cuenta ngrok gratuita: https://dashboard.ngrok.com/signup
- Auth Token de ngrok: https://dashboard.ngrok.com/get-started/your-authtoken

⚠️ Antes de empezar: **Runtime → Change runtime type → T4 GPU**

In [29]:
#===========================================================================
# 🧹 LIMPIEZA (ejecutar al terminar la sesión)
# ===========================================================================

from pyngrok import ngrok
ngrok.kill()
subprocess.run(['pkill', '-f', 'ollama'], capture_output=True)
print("✅ Túnel y servidor cerrados.")

✅ Túnel y servidor cerrados.


In [1]:
# ===========================================================================
# PASO 0: Verificar GPU
# ===========================================================================
# Siempre verificamos primero que tenemos GPU disponible.
# Sin GPU, los modelos tardan 10-20x más → clase inviable.

import subprocess

result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
if result.returncode != 0:
    raise RuntimeError(
        "❌ No se detectó GPU.\n"
        "Ve a Runtime → Change runtime type → T4 GPU y vuelve a ejecutar."
    )

print("✅ GPU disponible:")
print(result.stdout.split('\n')[8])  # Línea con el modelo de GPU

✅ GPU disponible:
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |


In [2]:

# ===========================================================================
# PASO 1: Instalar Ollama
# ===========================================================================
# Ollama es un servidor que gestiona modelos LLM localmente.
# Expone una API REST en el puerto 11434, compatible con la API de OpenAI.
#
# Dependencias del sistema:
#   - zstd: para descomprimir el binario de Ollama
#   - pciutils: para que Ollama detecte la GPU (¡sin esto corre en CPU!)

import os
os.environ['DEBIAN_FRONTEND'] = 'noninteractive'

print("📦 Instalando dependencias del sistema...")
!sudo apt-get update -qq 2>/dev/null
!sudo apt-get install -y -qq zstd pciutils 2>/dev/null

print("\n📥 Instalando Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh 2>&1 | grep -v 'systemd'

!ollama --version

📦 Instalando dependencias del sistema...
Selecting previously unselected package pci.ids.
(Reading database ... 121852 files and directories currently installed.)
Preparing to unpack .../pci.ids_0.0~2022.01.22-1ubuntu0.1_all.deb ...
Unpacking pci.ids (0.0~2022.01.22-1ubuntu0.1) ...
Selecting previously unselected package libpci3:amd64.
Preparing to unpack .../libpci3_1%3a3.7.0-6_amd64.deb ...
Unpacking libpci3:amd64 (1:3.7.0-6) ...
Selecting previously unselected package pciutils.
Preparing to unpack .../pciutils_1%3a3.7.0-6_amd64.deb ...
Unpacking pciutils (1:3.7.0-6) ...
Selecting previously unselected package zstd.
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up pci.ids (0.0~2022.01.22-1ubuntu0.1) ...
Setting up libpci3:amd64 (1:3.7.0-6) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Setting up pciutils (1:3.7.0-6) ...
Processing triggers for man-db (2.10.2-1) ...
Processing triggers for libc-bin (2.35-0ubuntu3.8) ...
/s

In [20]:
# ===========================================================================
# PASO 2: Instalar librerías Python
# ===========================================================================
# - pyngrok: cliente Python para el túnel ngrok
# - ollama: cliente oficial para la API de Ollama
# - langchain + langchain-ollama: framework de orquestación de LLMs
# - langchain-community: integraciones de terceros (vectorstores, loaders, etc.)
# - faiss-cpu: librería de búsqueda vectorial eficiente (de Meta)
#   Usamos la versión CPU porque FAISS corre en RAM, no en VRAM

print("📦 Instalando librerías Python...")
!pip install -qq pyngrok ollama
!pip install -qq langchain langchain-ollama langchain-community langchain-text-splitters
!pip install -qq faiss-cpu

📦 Instalando librerías Python...


In [4]:
# ===========================================================================
# PASO 3: Configurar ngrok
# ===========================================================================
# ngrok crea un túnel HTTPS entre internet y un puerto local de Colab.
# Necesitamos esto porque Colab no tiene IP pública directa.
#
# Configura tu token en Colab Secrets (icono 🔑 en el panel izquierdo):
#   Nombre: NGROK_AUTHTOKEN
#   Valor: tu token de https://dashboard.ngrok.com/get-started/your-authtoken

from google.colab import userdata

try:
    NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')
    print(f"✅ Token cargado: {NGROK_AUTHTOKEN[:2]}...{NGROK_AUTHTOKEN[-2:]}")
except Exception:
    raise ValueError(
        "❌ Token de ngrok no encontrado.\n"
        "Añádelo en Colab Secrets (🔑) con el nombre NGROK_AUTHTOKEN."
    )

✅ Token cargado: 39...8e


In [5]:
# ===========================================================================
# PASO 4: Iniciar el servidor Ollama
# ===========================================================================
# Lanzamos Ollama como proceso en background.
# OLLAMA_HOST=0.0.0.0 es necesario para que ngrok pueda alcanzarlo.

import subprocess, time, requests

os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'

print("🚀 Iniciando servidor Ollama...")
ollama_proc = subprocess.Popen(
    ['ollama', 'serve'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)
time.sleep(5)

# Verificar que el servidor responde
for intento in range(3):
    try:
        r = requests.get('http://localhost:11434', timeout=10)
        print(f"✅ Servidor activo: {r.text.strip()}")
        break
    except requests.ConnectionError:
        print(f"   Intento {intento + 1}/3... esperando")
        time.sleep(5)
else:
    raise RuntimeError("❌ El servidor Ollama no arrancó correctamente.")


🚀 Iniciando servidor Ollama...
✅ Servidor activo: Ollama is running


In [6]:

#===========================================================================
# PASO 5: Crear el túnel ngrok
# ===========================================================================
# host_header="localhost:11434" es CRÍTICO.
# Ollama solo acepta peticiones cuyo header "Host" sea localhost.
# Sin este parámetro, todas las peticiones vía ngrok serían rechazadas.

from pyngrok import ngrok

ngrok.kill()           # Cerrar túneles previos si los hubiera
time.sleep(2)
ngrok.set_auth_token(NGROK_AUTHTOKEN)

tunnel = ngrok.connect(
    addr='11434',
    proto='http',
    host_header='localhost:11434'
)
OLLAMA_URL = tunnel.public_url

print(f"\n🌐 Servidor expuesto en: {OLLAMA_URL}")
print(f"   Prueba en el navegador: {OLLAMA_URL}/api/tags")


🌐 Servidor expuesto en: https://mckinley-sly-wretchedly.ngrok-free.dev
   Prueba en el navegador: https://mckinley-sly-wretchedly.ngrok-free.dev/api/tags


In [7]:
#===========================================================================
# PASO 6: Descargar el modelo
# ===========================================================================
# Usamos qwen3:4b: ligero (~2.5 GB), rápido en T4, y muy capaz en español.
# Alternativas si quieres más calidad (a costa de velocidad):
#   - qwen3:8b  (~5 GB, excelente calidad)
#   - llama3.1:8b (~5 GB, referencia open-source)
#   - mistral:7b (~4 GB, muy eficiente)

MODEL = 'qwen3:4b'

print(f"📥 Descargando {MODEL} (puede tardar 2-5 minutos)...")
!ollama pull {MODEL}

print(f"\n✅ Modelos disponibles:")
!ollama list

📥 Descargando qwen3:4b (puede tardar 2-5 minutos)...


✅ Modelos disponibles:
NAME        ID              SIZE      MODIFIED               
qwen3:4b    359d7dd4bcda    2.5 GB    Less than a second ago    


In [9]:
#===========================================================================
# PASO 7: Verificar que el modelo usa GPU
# ===========================================================================
# Enviamos una petición corta para cargar el modelo en VRAM.
# Luego confirmamos con `ollama ps` que aparece "GPU" y no "CPU".
# Si falla aumentar el tiempo de espera (timeout)

import requests, json

print("🔥 Cargando modelo en GPU...")
r = requests.post(
    'http://localhost:11434/api/generate',
    json={"model": MODEL, "prompt": "Hola", "stream": False},
    timeout=120
)
print("   Modelo cargado.")

print("\n📊 Estado del modelo (debe decir GPU):")
!ollama ps

print("\n🖥️  Uso de VRAM:")
!nvidia-smi --query-gpu=name,memory.used,memory.total --format=csv,noheader

🔥 Cargando modelo en GPU...
   Modelo cargado.

📊 Estado del modelo (debe decir GPU):
NAME        ID              SIZE      PROCESSOR    CONTEXT    UNTIL              
qwen3:4b    359d7dd4bcda    3.6 GB    100% GPU     4096       4 minutes from now    

🖥️  Uso de VRAM:
Tesla T4, 3223 MiB, 15360 MiB


In [10]:
#===========================================================================
# PASO 8: Probar el LLM directamente (SIN RAG)
# ===========================================================================
# Antes de construir el RAG, veamos la limitación que queremos resolver:
# el modelo no conoce información privada o reciente.
#
# Preguntaremos sobre una empresa ficticia → el modelo inventará o dirá
# que no sabe. Esto motiva la necesidad del RAG.

import ollama as ollama_client

client = ollama_client.Client(host=OLLAMA_URL)

PREGUNTA_DEMO = "¿Cuáles son los productos que ofrece TechNova Solutions y cuál es su precio?"

print("=" * 60)
print("🤖 LLM SIN contexto (respuesta de base)")
print("=" * 60)
print(f"Pregunta: {PREGUNTA_DEMO}\n")

respuesta = client.generate(model=MODEL, prompt=PREGUNTA_DEMO)
print("Respuesta:")
print(respuesta['response'])
print("\n💡 Observación: el modelo no conoce TechNova Solutions.")
print("   Responderá que no tiene información o inventará datos.")
print("   Esto es lo que RAG viene a resolver.")

🤖 LLM SIN contexto (respuesta de base)
Pregunta: ¿Cuáles son los productos que ofrece TechNova Solutions y cuál es su precio?

Respuesta:
No tengo información específica sobre los productos y precios de TechNova Solutions en mi base de conocimientos. Es posible que esta empresa sea ficticia, tenga un nombre similar a otra compañía, o que los detalles no estén disponibles en los datos que he recibido. 

Para obtener información precisa, te recomiendo:
1. Visitar su **sitio web oficial**.
2. Contactar directamente a TechNova Solutions a través de su correo o teléfono.
3. Revisar plataformas de comercio electrónico o catálogos públicos si aplica.

Los precios suelen variar según el paquete, región, condiciones de uso y otros factores, por lo que siempre es importante confirmar con la fuente oficial. ¡Espero que te sea útil! 😊

💡 Observación: el modelo no conoce TechNova Solutions.
   Responderá que no tiene información o inventará datos.
   Esto es lo que RAG viene a resolver.


In [21]:
#===========================================================================
# PASO 9: Preparar los documentos (base de conocimiento)
# ===========================================================================
# En un caso real, estos documentos serían PDFs, páginas web, bases de datos, etc.
# Aquí usamos texto plano para mantener el ejemplo claro y reproducible.
#
# Concepto clave: los documentos se dividen en "chunks" (fragmentos).
# Cada chunk se convierte en un vector (embedding) para poder buscar
# por similitud semántica, no por palabras exactas.

from langchain_core.documents import Document

# Base de conocimiento: documentación interna de una empresa ficticia
documentos_raw = [
    Document(
        page_content="""
        TechNova Solutions - Catálogo de Productos 2024

        1. DataSync Pro
           Plataforma de sincronización de datos en tiempo real.
           Precio: 299€/mes por empresa (hasta 10 usuarios).
           Incluye: dashboard analítico, API REST, soporte 24/7.

        2. CloudGuard Enterprise
           Suite de seguridad cloud con detección de amenazas por IA.
           Precio: 599€/mes. Incluye auditorías semanales automatizadas.

        3. InsightBoard
           Herramienta de Business Intelligence con visualizaciones interactivas.
           Precio: 149€/mes (plan básico) o 349€/mes (plan avanzado con predicciones).
        """,
        metadata={"fuente": "catalogo_productos.pdf", "seccion": "precios"}
    ),
    Document(
        page_content="""
        TechNova Solutions - Política de Soporte Técnico

        Niveles de soporte disponibles:
        - Nivel Básico (incluido en todos los planes): tickets por email,
          tiempo de respuesta máximo 48 horas laborables.
        - Nivel Premium (50€/mes adicionales): chat en vivo y teléfono,
          tiempo de respuesta máximo 4 horas, gestor de cuenta dedicado.
        - Nivel Enterprise (incluido en CloudGuard): soporte 24/7,
          tiempo de respuesta máximo 1 hora, ingeniero de soporte asignado.

        Para escalar un ticket a nivel crítico, enviar email a:
        soporte-critico@technova.es con asunto [CRÍTICO].
        """,
        metadata={"fuente": "politica_soporte.pdf", "seccion": "soporte"}
    ),
    Document(
        page_content="""
        TechNova Solutions - Preguntas Frecuentes (FAQ)

        P: ¿Puedo probar los productos antes de comprar?
        R: Sí. Todos los productos ofrecen período de prueba gratuito de 14 días,
           sin tarjeta de crédito requerida. Accede desde technova.es/trial.

        P: ¿Hay descuentos para startups o ONGs?
        R: Sí. Ofrecemos 40% de descuento para startups de menos de 2 años
           y 50% para ONGs registradas. Contactar a ventas@technova.es.

        P: ¿Los datos se almacenan en Europa?
        R: Sí. Todos los datos se almacenan en centros de datos certificados
           ISO 27001 ubicados en Frankfurt y Madrid, cumpliendo con el RGPD.

        P: ¿Se puede integrar con herramientas existentes como Slack o Jira?
        R: DataSync Pro e InsightBoard tienen integraciones nativas con Slack,
           Jira, Notion y más de 50 herramientas via Zapier.
        """,
        metadata={"fuente": "faq.pdf", "seccion": "preguntas_frecuentes"}
    ),
]

print(f"📄 {len(documentos_raw)} documentos cargados.")
print(f"   Fuentes: {[d.metadata['fuente'] for d in documentos_raw]}")

📄 3 documentos cargados.
   Fuentes: ['catalogo_productos.pdf', 'politica_soporte.pdf', 'faq.pdf']


In [23]:
#===========================================================================
# PASO 10: Dividir documentos en chunks
# ===========================================================================
# ¿Por qué dividir? Los LLMs tienen límite de contexto (tokens que pueden
# "leer" a la vez). Además, recuperar chunks pequeños y precisos es más
# efectivo que enviar documentos enteros.
#
# chunk_size: tamaño máximo de cada fragmento (en caracteres)
# chunk_overlap: solapamiento entre chunks para no perder contexto en los bordes

from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,       # Cada chunk: hasta 500 caracteres
    chunk_overlap=50,     # Los últimos 50 chars de un chunk se repiten al inicio del siguiente
    separators=["\n\n", "\n", ". ", " "]  # Prioridad de separación: párrafos > líneas > frases
)

chunks = splitter.split_documents(documentos_raw)

print(f"✂️  Documentos divididos en {len(chunks)} chunks.")
print(f"\n   Ejemplo — Chunk #1:")
print(f"   Fuente: {chunks[0].metadata['fuente']}")
print(f"   Texto: {chunks[0].page_content[:200]}...")

✂️  Documentos divididos en 7 chunks.

   Ejemplo — Chunk #1:
   Fuente: catalogo_productos.pdf
   Texto: TechNova Solutions - Catálogo de Productos 2024

        1. DataSync Pro
           Plataforma de sincronización de datos en tiempo real.
           Precio: 299€/mes por empresa (hasta 10 usuarios).
 ...


In [24]:
#===========================================================================
# PASO 11: Crear embeddings y la base vectorial
# ===========================================================================
# Los embeddings son representaciones numéricas del significado de un texto.
# Textos semánticamente similares tienen vectores cercanos en el espacio.
#
# Usamos OllamaEmbeddings con nomic-embed-text: modelo ligero (~274 MB)
# diseñado específicamente para embeddings, mucho más eficiente que usar
# un modelo de chat para esto.
#
# FAISS (Facebook AI Similarity Search) almacena los vectores en memoria
# y permite búsquedas de similitud muy rápidas (algoritmo HNSW).

from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# Descargar el modelo de embeddings
print("📥 Descargando modelo de embeddings (nomic-embed-text, ~274 MB)...")
!ollama pull nomic-embed-text

# Crear el objeto de embeddings
embeddings = OllamaEmbeddings(
    base_url=OLLAMA_URL,
    model="nomic-embed-text"
)

# Crear la base vectorial indexando todos los chunks
# Este proceso: texto → embedding (vector) → almacenado en FAISS
print("\n🔢 Generando embeddings e indexando chunks...")
vectorstore = FAISS.from_documents(chunks, embeddings)

print(f"✅ Base vectorial creada con {vectorstore.index.ntotal} vectores.")
print(f"   Dimensión de cada vector: {vectorstore.index.d}")

📥 Descargando modelo de embeddings (nomic-embed-text, ~274 MB)...


🔢 Generando embeddings e indexando chunks...
✅ Base vectorial creada con 7 vectores.
   Dimensión de cada vector: 768


In [25]:
#===========================================================================
# PASO 12: Probar el recuperador (Retriever)
# ===========================================================================
# El retriever es el componente que, dada una pregunta, busca los chunks
# más relevantes en la base vectorial por similitud semántica.
#
# Esto funciona porque la pregunta también se convierte en un vector,
# y se buscan los vectores de chunks más cercanos (distancia coseno).

retriever = vectorstore.as_retriever(
    search_type="similarity",   # Búsqueda por similitud coseno
    search_kwargs={"k": 2}      # Recuperar los 2 chunks más relevantes
)

# Probemos manualmente el retriever
pregunta_test = "¿Cuánto cuesta el plan de soporte premium?"

docs_recuperados = retriever.invoke(pregunta_test)

print("=" * 60)
print("🔍 TEST DEL RECUPERADOR")
print("=" * 60)
print(f"Pregunta: {pregunta_test}")
print(f"\nChunks recuperados ({len(docs_recuperados)}):")
for i, doc in enumerate(docs_recuperados, 1):
    print(f"\n  [{i}] Fuente: {doc.metadata['fuente']}")
    print(f"       Texto: {doc.page_content[:300]}...")

print("\n💡 El retriever encontró el chunk relevante sin buscar por palabras exactas.")
print("   Probó similitud semántica: 'soporte premium' ≈ 'Nivel Premium'.")

🔍 TEST DEL RECUPERADOR
Pregunta: ¿Cuánto cuesta el plan de soporte premium?

Chunks recuperados (2):

  [1] Fuente: politica_soporte.pdf
       Texto: Niveles de soporte disponibles:
        - Nivel Básico (incluido en todos los planes): tickets por email,
          tiempo de respuesta máximo 48 horas laborables.
        - Nivel Premium (50€/mes adicionales): chat en vivo y teléfono,
          tiempo de respuesta máximo 4 horas, gestor de cuenta d...

  [2] Fuente: catalogo_productos.pdf
       Texto: 3. InsightBoard
           Herramienta de Business Intelligence con visualizaciones interactivas.
           Precio: 149€/mes (plan básico) o 349€/mes (plan avanzado con predicciones)....

💡 El retriever encontró el chunk relevante sin buscar por palabras exactas.
   Probó similitud semántica: 'soporte premium' ≈ 'Nivel Premium'.


In [27]:
#===========================================================================
# PASO 13: Construir la cadena RAG con LangChain
# ===========================================================================
# Aquí ensamblamos el pipeline completo. LangChain usa el paradigma LCEL
# (LangChain Expression Language) con el operador | para encadenar pasos.
#
# Pipeline:
#   {"context": retriever, "question": passthrough}
#        ↓
#   prompt_template  (inserta context + question en el prompt)
#        ↓
#   llm              (genera la respuesta)
#        ↓
#   StrOutputParser  (extrae el texto de la respuesta)


from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. Conectar el LLM de Ollama
llm = ChatOllama(
    base_url=OLLAMA_URL,
    model=MODEL,
    temperature=0.1,    # Temperatura baja: respuestas más precisas y consistentes
    num_predict=512,    # Máximo de tokens a generar
)

# 2. Definir el prompt
# El prompt le indica al LLM cómo debe usar el contexto recuperado.
# {context} y {question} son variables que se rellenan en cada llamada.
PROMPT_TEMPLATE = """
Eres un asistente de atención al cliente de TechNova Solutions.
Responde la pregunta del usuario basándote ÚNICAMENTE en el contexto proporcionado.
Si la información no está en el contexto, di exactamente: "No tengo esa información disponible."
No inventes datos ni precios.

Contexto:
{context}

Pregunta: {question}

Respuesta:"""

prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

# 3. Función auxiliar para formatear los chunks recuperados
def formatear_docs(docs):
    """Concatena el contenido de los chunks recuperados en un solo string."""
    return "\n\n---\n\n".join(doc.page_content for doc in docs)

# 4. Ensamblar la cadena RAG con LCEL
cadena_rag = (
    {
        "context": retriever | formatear_docs,   # Recuperar + formatear
        "question": RunnablePassthrough()         # Pasar la pregunta tal cual
    }
    | prompt           # Construir el prompt con context + question
    | llm              # Llamar al LLM
    | StrOutputParser() # Extraer solo el texto de la respuesta
)

print("✅ Cadena RAG construida:")
print("   Pregunta → Retriever → Chunks → Prompt → LLM → Respuesta")

✅ Cadena RAG construida:
   Pregunta → Retriever → Chunks → Prompt → LLM → Respuesta


In [28]:
#===========================================================================
# PASO 14: Probar el sistema RAG
# ===========================================================================
# Ahora comparamos la misma pregunta del Paso 8 con y sin RAG.
# La diferencia debe ser notable: con RAG el modelo responde con datos reales.

def preguntar_con_rag(pregunta: str) -> str:
    """Envía una pregunta al sistema RAG y muestra el resultado."""
    print("=" * 60)
    print(f"❓ Pregunta: {pregunta}")
    print("=" * 60)

    # Mostrar qué chunks se están recuperando (transparencia del proceso)
    docs = retriever.invoke(pregunta)
    print(f"\n🔍 Chunks recuperados ({len(docs)}):")
    for i, doc in enumerate(docs, 1):
        print(f"   [{i}] {doc.metadata['fuente']} — {doc.page_content[:100].strip()}...")

    # Generar la respuesta
    print("\n💬 Respuesta del LLM con contexto:\n")
    respuesta = cadena_rag.invoke(pregunta)
    print(respuesta)
    print()
    return respuesta


# Batería de preguntas de prueba
preguntas = [
    "¿Cuáles son los productos que ofrece TechNova Solutions y cuál es su precio?",
    "¿Ofrecen descuentos para ONGs?",
    "¿Cómo puedo escalar un ticket urgente al soporte?",
    "¿Cuál es la capital de Francia?",   # Pregunta fuera del contexto → debe decir que no sabe
]

for pregunta in preguntas:
    preguntar_con_rag(pregunta)
    print()

❓ Pregunta: ¿Cuáles son los productos que ofrece TechNova Solutions y cuál es su precio?

🔍 Chunks recuperados (2):
   [1] faq.pdf — TechNova Solutions - Preguntas Frecuentes (FAQ)

        P: ¿Puedo probar los productos antes de com...
   [2] politica_soporte.pdf — Niveles de soporte disponibles:
        - Nivel Básico (incluido en todos los planes): tickets por e...

💬 Respuesta del LLM con contexto:

No tengo esa información disponible.


❓ Pregunta: ¿Ofrecen descuentos para ONGs?

🔍 Chunks recuperados (2):
   [1] faq.pdf — TechNova Solutions - Preguntas Frecuentes (FAQ)

        P: ¿Puedo probar los productos antes de com...
   [2] politica_soporte.pdf — Niveles de soporte disponibles:
        - Nivel Básico (incluido en todos los planes): tickets por e...

💬 Respuesta del LLM con contexto:




❓ Pregunta: ¿Cómo puedo escalar un ticket urgente al soporte?

🔍 Chunks recuperados (2):
   [1] politica_soporte.pdf — Para escalar un ticket a nivel crítico, enviar email a:
        soporte

In [ ]:
#===========================================================================
# 🎯 RESUMEN DE LO APRENDIDO
# ===========================================================================
"""
En esta clase construiste un sistema RAG completo:

┌─────────────────────────────────────────────────────────────┐
│                    ARQUITECTURA RAG                         │
│                                                             │
│  TUS DOCS → Chunks → Embeddings → FAISS (base vectorial)   │
│                                        ↑                   │
│  Pregunta ──────────────────────────── Retriever           │
│      │                                     │               │
│      │              Chunks relevantes ◄────┘               │
│      │                    │                                 │
│      └──► Prompt ◄────────┘                                 │
│               │                                             │
│              LLM (Ollama / qwen3:4b en T4)                  │
│               │                                             │
│          Respuesta fundamentada en tus documentos           │
└─────────────────────────────────────────────────────────────┘

Componentes LangChain usados:
  • RecursiveCharacterTextSplitter  → dividir documentos en chunks (langchain-text-splitters)
  • OllamaEmbeddings               → convertir texto en vectores
  • FAISS                          → base vectorial para búsqueda
  • ChatOllama                     → interfaz con el LLM
  • ChatPromptTemplate             → plantilla de prompt estructurada
  • LCEL ( | )                     → encadenar componentes

Próximos pasos para explorar:
  • Usar PDFs reales (PyPDFLoader, WebBaseLoader)
  • Persistir FAISS en Google Drive para reutilizar entre sesiones
  • Añadir memoria conversacional (historial de chat)
  • Usar Chroma o Qdrant como alternativas a FAISS para producción
  • Evaluar la calidad del RAG (RAGAS framework)
"""

print("🎓 ¡Clase completada!")
print("   Tienes un sistema RAG funcional con LLM local, embeddings y búsqueda vectorial.")